# 6일차: PyTorch 텐서와 Autograd 기본

## 학습 목표
- PyTorch 텐서 기본 사용법 학습
- Autograd(자동 미분) 메커니즘 이해
- Gradient 계산 및 역전파 실습
- Gradient 추적 제어하기

In [ ]:
import torch
import numpy as np
print(f'PyTorch 버전: {torch.__version__}')

## 💡 이론 개요

### PyTorch Tensor
- NumPy와 유사한 다차원 배열
- **GPU 연산 지원**
- **자동 미분 지원** (autograd)

### Autograd (자동 미분)
- 연산 그래프 자동 생성
- `.backward()` 호출로 gradient 계산
- 딥러닝 역전파의 핵심

### 주요 개념
- `requires_grad=True`: gradient 추적 시작
- `.backward()`: gradient 계산
- `.grad`: 계산된 gradient 저장

## 📚 1. PyTorch 텐서 기본

### 1.1 텐서 생성

In [ ]:
# 리스트로부터 텐서 생성
t = torch.tensor([1, 2, 3, 4, 5])
print('텐서:', t)
print(f'타입: {type(t)}')
print(f'dtype: {t.dtype}')
print(f'shape: {t.shape}')
print(f'device: {t.device}')  # cpu 또는 cuda
print()

In [ ]:
# 다양한 텐서 생성 방법
zeros = torch.zeros(2, 3)
ones = torch.ones(3, 2)
rand = torch.rand(2, 2)  # [0, 1) 균등분포
randn = torch.randn(2, 2)  # 표준정규분포

print('zeros (2×3):'); print(zeros)
print('\nones (3×2):'); print(ones)
print('\nrand (2×2):'); print(rand)
print('\nrandn (2×2):'); print(randn)

### 1.2 NumPy와 상호 변환

In [ ]:
# NumPy → PyTorch
np_array = np.array([[1, 2], [3, 4]])
torch_tensor = torch.from_numpy(np_array)

print('NumPy 배열:')
print(np_array)
print('\nPyTorch 텐서:')
print(torch_tensor)
print()

# PyTorch → NumPy
back_to_numpy = torch_tensor.numpy()
print('다시 NumPy로:')
print(back_to_numpy)
print()

print('⚠️ 주의: 메모리를 공유합니다!')
np_array[0, 0] = 999
print('NumPy 수정 후 텐서:', torch_tensor)

## 📚 2. Autograd 기본

### 2.1 간단한 미분

In [ ]:
# y = x² + 3x + 5에서 dy/dx 계산
x = torch.tensor(2.0, requires_grad=True)
print(f'x = {x}')
print(f'requires_grad: {x.requires_grad}')
print()

# Forward pass
y = x**2 + 3*x + 5
print(f'y = x² + 3x + 5 = {y.item()}')
print()

# Backward pass (gradient 계산)
y.backward()

print('Gradient 계산 완료!')
print(f'dy/dx = 2x + 3 = 2×{x.item()} + 3 = {x.grad.item()}')
print()
print(f'실제 gradient: {x.grad.item()}')

### 2.2 다변수 함수의 gradient

In [ ]:
# z = x² + 2xy + y²에서 ∂z/∂x, ∂z/∂y
x = torch.tensor(1.0, requires_grad=True)
y = torch.tensor(2.0, requires_grad=True)

print(f'x = {x.item()}, y = {y.item()}')
print()

# Forward
z = x**2 + 2*x*y + y**2
print(f'z = x² + 2xy + y² = {z.item()}')
print()

# Backward
z.backward()

print('Gradients:')
print(f'∂z/∂x = 2x + 2y = 2×{x.item()} + 2×{y.item()} = {x.grad.item()}')
print(f'∂z/∂y = 2x + 2y = 2×{x.item()} + 2×{y.item()} = {y.grad.item()}')
print()
print('수식: ∂z/∂x = ∂z/∂y = 2x + 2y')

### 2.3 벡터 및 행렬 연산

In [ ]:
# 벡터 내적의 gradient
v = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)
w = torch.tensor([4.0, 5.0, 6.0])

print(f'v = {v}')
print(f'w = {w}')
print()

# 내적: v·w
dot_product = torch.dot(v, w)
print(f'v·w = {dot_product.item()}')
print()

dot_product.backward()

print('∂(v·w)/∂v = w')
print(f'gradient: {v.grad}')
print(f'w와 일치: {torch.allclose(v.grad, w)}')

## 📚 3. Gradient 제어

### 3.1 torch.no_grad()

In [ ]:
x = torch.tensor([1.0, 2.0], requires_grad=True)

print('일반 연산 (gradient 추적):')
y = x * 2
print(f'y.requires_grad: {y.requires_grad}')
print()

print('torch.no_grad() 사용:')
with torch.no_grad():
    z = x * 2
    print(f'z.requires_grad: {z.requires_grad}')
print()
print('⚠️ Inference 시 메모리 절약을 위해 no_grad 사용!')

### 3.2 Gradient 누적과 초기화

In [ ]:
x = torch.tensor(3.0, requires_grad=True)

# 첫 번째 backward
y1 = x ** 2
y1.backward()
print(f'첫 번째 backward 후 x.grad: {x.grad}')

# gradient 초기화 없이 두 번째 backward
y2 = x ** 3
y2.backward()
print(f'두 번째 backward 후 x.grad: {x.grad}')
print('→ Gradient가 누적됨!')
print()

# Gradient 초기화
x.grad.zero_()
print(f'초기화 후 x.grad: {x.grad}')
print()
print('💡 학습 루프에서는 매 iteration마다 optimizer.zero_grad() 필요!')

## 📚 4. 실전 예제: 선형 회귀

### 4.1 데이터 준비

In [ ]:
# 데이터: y = 3x + 1 + noise
torch.manual_seed(42)
x_data = torch.randn(100, 1)
y_data = 3 * x_data + 1 + 0.1 * torch.randn(100, 1)

print('데이터 shape:')
print(f'x: {x_data.shape}')
print(f'y: {y_data.shape}')
print()
print('목표: y = 3x + 1을 학습으로 찾기')

### 4.2 모델 파라미터

In [ ]:
# 파라미터 초기화
w = torch.randn(1, 1, requires_grad=True)
b = torch.zeros(1, 1, requires_grad=True)

print(f'초기 w: {w.item():.4f}')
print(f'초기 b: {b.item():.4f}')

### 4.3 학습 루프

In [ ]:
learning_rate = 0.01
num_epochs = 100

for epoch in range(num_epochs):
    # Forward pass
    y_pred = x_data @ w + b
    
    # Loss (MSE)
    loss = torch.mean((y_pred - y_data) ** 2)
    
    # Backward pass
    loss.backward()
    
    # Update (gradient descent)
    with torch.no_grad():
        w -= learning_rate * w.grad
        b -= learning_rate * b.grad
    
    # Gradient 초기화
    w.grad.zero_()
    b.grad.zero_()
    
    if (epoch + 1) % 20 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, w: {w.item():.4f}, b: {b.item():.4f}')

print()
print(f'최종 w: {w.item():.4f} (목표: 3.0)')
print(f'최종 b: {b.item():.4f} (목표: 1.0)')

## 🔥 연습 문제

### 문제 1: 단일 변수 미분

In [ ]:
# TODO: f(x) = x³ - 2x² + 5x - 1에서
# x = 2일 때 df/dx를 계산하세요

# 여기에 코드 작성


In [ ]:
# 해답
x = torch.tensor(2.0, requires_grad=True)
f = x**3 - 2*x**2 + 5*x - 1

f.backward()

print(f'f(x) = x³ - 2x² + 5x - 1')
print(f'f\'(x) = 3x² - 4x + 5')
print(f'\nx = 2일 때:')
print(f'f(2) = {f.item()}')
print(f'f\'(2) = 3×4 - 4×2 + 5 = 9')
print(f'계산된 gradient: {x.grad.item()}')

### 문제 2: 벡터 함수

In [ ]:
# TODO: x = [1, 2, 3]일 때
# f(x) = Σxᵢ²의 gradient를 구하세요

# 여기에 코드 작성한


In [ ]:
# 해답
x = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)
f = torch.sum(x ** 2)

f.backward()

print(f'f(x) = Σxᵢ² = x₁² + x₂² + x₃²')
print(f'∂f/∂xᵢ = 2xᵢ')
print(f'\nx = [1, 2, 3]')
print(f'gradient = [2×1, 2×2, 2×3] = {x.grad}')

## 📝 복습

### 핵심 정리

**PyTorch Tensor**
- GPU 연산 가능 (`.to('cuda')`)
- NumPy 호환 (`.numpy()`, `torch.from_numpy()`)
- 자동 미분 지원

**Autograd**
- `requires_grad=True`: 추적 시작
- `.backward()`: gradient 계산
- `.grad`: gradient 값
- `zero_()`: gradient 초기화

**주의사항**
1. Gradient 누적 - 매번 초기화 필요
2. Inference 시 `torch.no_grad()` 사용
3. `.item()`으로 스칼라 값 추출

### 다음 학습
Day 7: nn.Module, 손실함수, 옵티마이저로 완전한 학습 파이프라인 구축